# Imports and function definitions

In [ ]:
%matplotlib inline
from os.path import basename, splitext
import glob
import re

import numpy as np
import matplotlib.pyplot as plt
import typhon.plots
from netCDF4 import Dataset


plt.style.use(typhon.plots.styles('typhon'))


def extract_scale(filename):
    """Extract the CO2 increase from filename.
    
    Parameters:
        filename (str): Path to netCDF file.
        
    Returns:
        str: CO2 factor.
        
    Examples:
        >>> extract_scale('results/tropical_co2_x0.5-conv_fVMR.nc')
        '0.5'
    """
    pattern = re.compile('.*co2_x(.*)-.*\.nc')  
    matches = pattern.match(filename)

    if matches:
        return matches.group(1)
    else:
        return filename
    
    
def running_mean(x, N):
    """Calculate a running mean over a given data array.
    
    Paramters:
        x (ndarray): Data array.
        N (int): Window size.
    
    Return:
        ndarray: Running mean.
    """
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 


def calculate_toa(data):
    """Calculate radiation budget at top of the atmosphere (TOA).
    
    Parameters:
        data (dict): Dictionary containing short- and longwave fluxes.
        
    Returns:
        ndarray: Radiation budget at TOA.
    """
    lw_flxu = data.variables['lw_flxu'][:, -1]
    lw_flxd = data.variables['lw_flxd'][:, -1]
    sw_flxu = data.variables['sw_flxu'][:, -1]
    sw_flxd = data.variables['sw_flxd'][:, -1]
    return ((sw_flxd + lw_flxd) - (sw_flxu + lw_flxu))

# Surface temperature and radiation budget

In [ ]:
season = 'tropical'
scenarios = sorted(
    glob.glob(f'results/diurnalcycle/{season}_co2_x*-diurnalcycle.nc'),
    reverse=True,
)

fig, [ax, ax2] = plt.subplots(nrows=2, sharex=True)
fig.suptitle('FASCOD {season}'.format(season=season.capitalize()))
for scenario in scenarios:
    data = Dataset(scenario)
    print(f'Read {scenario}...')
    time = data.variables['time'][:] / 24  # time in days
    
    # Read and plot surface temperatures.
    Ts = data.variables['temperature'][:]
    ax.plot(
        time, Ts,
        label=r'$CO_2 \times {}$'.format(extract_scale(scenario)),
        linewidth=3,
    )
    
    # Calculate and plot radiation budget at top of the atmosphere.
    toa = calculate_toa(data)
    ax2.plot(
        time, toa,
        linewidth=3,
    )
    
ax.grid('on')
ax2.grid('on')
ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
ax2.set_xlabel('Day')
ax.set_xlim(left=0)
ax.set_ylabel('$T_s$ [K]')
ax2.set_ylabel('TOA [$Wm^{-2}$]')
# fig.savefig('plots/t_surface_double_co2_{season}_diurnalcycle_convective.pdf'.format(season=season))

## Running mean over radiation budget

In [ ]:
average_window = 100*8

fig, ax = plt.subplots()
for scenario in scenarios:
    data = Dataset(scenario)
    ax.plot(
        time[:-average_window+1],
        running_mean(calculate_toa(data), average_window),
        linewidth=3,
        label=r'$CO_2 \times {}$'.format(extract_scale(scenario)),
    )
ax.legend()
ax.grid(True)
ax.set_xlim(left=0)
ax.set_xlabel('Day')
ax.set_ylabel('TOA [$Wm^{-2}$]')
# fig.savefig('plots/toa_runningmean_double_co2_{season}_diurnalcycle_convective.pdf'.format(season=season))

# Plot equilibrium temperature profiles

In [ ]:
fig, ax = plt.subplots(figsize=typhon.plots.figsize(6, portrait=True))
for scenario in scenarios:
    data = Dataset(scenario)
    p = data.variables['plev'][:]
    T = data.variables['T'][-1, :]
    ci = np.argmin(T)
    
    l = typhon.plots.profile_p_log(
        p, T,
        label=r'$CO_2 \times {}$'.format(extract_scale(scenario)),
        linewidth=3,
        )
    ax.axhline(p[ci], linestyle='dotted', color=l[0].get_color(), alpha=0.3)
    
ax.set_title('FASCOD {season}'.format(season=season.capitalize()))
ax.grid('on')
ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
ax.set_xlabel('Temperature [K]')
ax.set_ylim(top=p.min(), bottom=p.max())
ax.set_xlim(left=180, right=320)
# fig.savefig('plots/t_profile_double_co2_{season}_diurnalcycle_convective.pdf'.format(season=season))

# Plot changes in water vapor mixing ratios

In [ ]:
scenario = scenarios[-1]

fig, ax = plt.subplots()
scale = extract_scale(scenario)
fig.suptitle(rf'FASCOD {season.capitalize()} | ${scale} \times CO_2$')
data = Dataset(scenario)
time = data.variables['time'][:] / 24  # time in days

# Calculate relative humidity.
q = data.variables['H2O'][:]
p = data.variables['plev'][:]
T = data.variables['T'][:]
RH = 100* typhon.atmosphere.relative_humidity(q, p, T)

sm = ax.pcolormesh(
    time, p, RH.T,
    cmap=plt.get_cmap('density', 10),
    rasterized=True,
    vmin=0,
    vmax=100,
)
cb = fig.colorbar(sm, label='Relative humidity [%]')
cb.set_ticks(np.arange(0, 101, 10))
ax.invert_yaxis()
ax.set_yscale('log')
ax.set_ylabel('Pressure [hPa]')
ax.set_ylim(p.max(), p.min())
ax.set_xlabel('Days')
ax.yaxis.set_major_formatter(typhon.plots.HectoPascalLogFormatter())
ax.yaxis.set_minor_formatter(typhon.plots.HectoPascalLogFormatter())


fig, ax = plt.subplots(figsize=(4, 6))
for scenario in scenarios:
    typhon.plots.profile_p_log(
        p, np.std(RH, axis=0),
        linewidth=3,
        label=r'$CO_2 \times {}$'.format(extract_scale(scenario)),
        )
ax.legend()
ax.grid('on')
ax.set_xlim(left=-0.01, right=0.15)
ax.set_ylim(p.max(), p.min())